In [ ]:
from google.colab import drive
import os
import csv
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', None)

### 0. Load Data From Drive

In [ ]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
DATA_DIR = '/content/drive/My Drive/NEU/Graduate/CS_6140/Data/raw_data/'
SAVE_DATA_DIR = '/content/drive/My Drive/NEU/Graduate/CS_6140/Data/'
ERRORS_F = 'PdM_errors.csv'
FAILURES_F = 'PdM_failures.csv'
MACHINES_F = 'PdM_machines.csv'
MAINT_F = 'PdM_maint.csv'
TELEMETRY_F = 'PdM_telemetry.csv'

ERRORS_F = os.path.join(DATA_DIR, ERRORS_F)
FAILURES_F = os.path.join(DATA_DIR, FAILURES_F)
MACHINES_F = os.path.join(DATA_DIR, MACHINES_F)
MAINT_F = os.path.join(DATA_DIR, MAINT_F)
TELEMETRY_F = os.path.join(DATA_DIR, TELEMETRY_F)

### 1. Telemetry File Week Numbers

In [ ]:
tel_df = pd.read_csv(
    TELEMETRY_F,
)
tel_df['machineID'] = tel_df['machineID'].astype(int)
tel_df['datetime'] = pd.to_datetime(tel_df['datetime'])
tel_df['volt'] = tel_df['volt'].astype(float)
tel_df['rotate'] = tel_df['rotate'].astype(float)
tel_df['pressure'] = tel_df['pressure'].astype(float)
tel_df['vibration'] = tel_df['vibration'].astype(float)

print(f"- {format(len(tel_df), ',')} total rows in tel_df.")
tel_df.head(10)

- 876,100 total rows in tel_df.


,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511
5,2015-01-01 11:00:00,1,172.504839,430.323362,95.927042,35.655017
6,2015-01-01 12:00:00,1,156.556031,499.071623,111.755684,42.753920
7,2015-01-01 13:00:00,1,172.522781,409.624717,101.001083,35.482009
8,2015-01-01 14:00:00,1,175.324524,398.648781,110.624361,45.482287
9,2015-01-01 15:00:00,1,169.218423,460.850670,104.848230,39.901735


In [ ]:
# sort by machineID and datetime
tel_df = tel_df.sort_values(
    by = ['machineID', 'datetime']
)


### 1.1) 12-, 24-, 48-, and 72-Hour Window Keys

In [ ]:
HOUR_WINDOWS_LIST = [12, 24, 48, 72]

for hour_window in HOUR_WINDOWS_LIST:

    # Calculating machineID_time_window
    tel_df[f'{hour_window}_machineID_time_window'] = tel_df.groupby('machineID')['datetime'].transform(
        lambda x: ((x - x.min()).dt.total_seconds() // 3_600 // hour_window) + 1
    )
    tel_df[f'{hour_window}_machineID_time_window'] = tel_df[f'{hour_window}_machineID_time_window'].astype(int)

    # Calculating order_in_time_window within each machineID and machineID_time_window group
    tel_df[f'{hour_window}_order_in_time_window'] = tel_df.groupby(
        ['machineID', f'{hour_window}_machineID_time_window']
    ).cumcount() + 1

    tel_df[f'{hour_window}_machine_example_ID'] = "(" + tel_df['machineID'].astype(str) + ", " + tel_df[f'{hour_window}_machineID_time_window'].astype(str) + ")"

    # create unique ID consisting of: machineID, machineID_time_window, order_in_time_window
    tel_df[f'{hour_window}_step_ID'] = "(" + tel_df['machineID'].astype(str) + ", " + tel_df[f'{hour_window}_machineID_time_window'].astype(str) + ", " + tel_df[f'{hour_window}_order_in_time_window'].astype(str) + ")"

    assert len(tel_df) == tel_df[f'{hour_window}_step_ID'].nunique()

### 1.2) Print Result

In [ ]:
print(f"- {format(len(tel_df), ',')} total rows in tel_df.")
tel_df.head(80)

- 876,100 total rows in tel_df.


,datetime,machineID,volt,rotate,pressure,vibration,12_machineID_time_window,12_order_in_time_window,12_machine_example_ID,12_step_ID,24_machineID_time_window,24_order_in_time_window,24_machine_example_ID,24_step_ID,48_machineID_time_window,48_order_in_time_window,48_machine_example_ID,48_step_ID,72_machineID_time_window,72_order_in_time_window,72_machine_example_ID,72_step_ID
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)"
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)"
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)"
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,1,4,"(1, 1)","(1, 1, 4)",1,4,"(1, 1)","(1, 1, 4)",1,4,"(1, 1)","(1, 1, 4)",1,4,"(1, 1)","(1, 1, 4)"
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,1,5,"(1, 1)","(1, 1, 5)",1,5,"(1, 1)","(1, 1, 5)",1,5,"(1, 1)","(1, 1, 5)",1,5,"(1, 1)","(1, 1, 5)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2015-01-04 09:00:00,1,157.436263,438.091311,113.100915,53.695544,7,4,"(1, 7)","(1, 7, 4)",4,4,"(1, 4)","(1, 4, 4)",2,28,"(1, 2)","(1, 2, 28)",2,4,"(1, 2)","(1, 2, 4)"
76,2015-01-04 10:00:00,1,153.143558,440.162685,94.524894,57.411078,7,5,"(1, 7)","(1, 7, 5)",4,5,"(1, 4)","(1, 4, 5)",2,29,"(1, 2)","(1, 2, 29)",2,5,"(1, 2)","(1, 2, 5)"
77,2015-01-04 11:00:00,1,215.656488,458.097746,95.036280,51.647981,7,6,"(1, 7)","(1, 7, 6)",4,6,"(1, 4)","(1, 4, 6)",2,30,"(1, 2)","(1, 2, 30)",2,6,"(1, 2)","(1, 2, 6)"
78,2015-01-04 12:00:00,1,173.525320,421.728389,100.617527,50.458297,7,7,"(1, 7)","(1, 7, 7)",4,7,"(1, 4)","(1, 4, 7)",2,31,"(1, 2)","(1, 2, 31)",2,7,"(1, 2)","(1, 2, 7)"


In [ ]:
for hour_window in HOUR_WINDOWS_LIST:
    print(tel_df[f'{hour_window}_order_in_time_window'].describe())
    print("\n")

count    876100.000000
mean          6.499372
std           3.452358
min           1.000000
25%           3.000000
50%           6.000000
75%           9.000000
max          12.000000
Name: 12_order_in_time_window, dtype: float64


count    876100.000000
mean         12.498687
std           6.922886
min           1.000000
25%           6.000000
50%          12.000000
75%          18.000000
max          24.000000
Name: 24_order_in_time_window, dtype: float64


count    876100.000000
mean         24.467184
std          13.852579
min           1.000000
25%          12.000000
50%          24.000000
75%          36.000000
max          48.000000
Name: 48_order_in_time_window, dtype: float64


count    876100.000000
mean         36.435681
std          20.769102
min           1.000000
25%          18.000000
50%          36.000000
75%          54.000000
max          72.000000
Name: 72_order_in_time_window, dtype: float64




### 2. Load Maintenance File and Merge onto Telemetry DataFrame

In [ ]:
maint_df = pd.read_csv(
    MAINT_F,
)
maint_df['machineID'] = maint_df['machineID'].astype(int)
maint_df['datetime'] = pd.to_datetime(maint_df['datetime'])
maint_df['comp'] = maint_df['comp'].astype(str)

print(f"- {format(len(maint_df), ',')} total rows in maint_df.")
maint_df.head(3)

- 3,286 total rows in maint_df.


,datetime,machineID,comp
0,2014-06-01 06:00:00,1,comp2
1,2014-07-16 06:00:00,1,comp4
2,2014-07-31 06:00:00,1,comp3


Because the maintenance CSV contains both routine maintenances and failures (captured in failures CSV), we must first remove the failures from `maint_df`.

In [ ]:
# loading failures data
failures_df = pd.read_csv(
    FAILURES_F,
)
failures_df['machineID'] = failures_df['machineID'].astype(int)
failures_df['datetime'] = pd.to_datetime(failures_df['datetime'])
failures_df['failure'] = failures_df['failure'].astype(str)

# `maint_df` columns: datetime, machineID, comp
# `failures_df` columns: datetime, machineID, failure
failures_df.rename(
    columns = {
        'failure': 'comp'
    },
    inplace = True
)

# removing rows from `maint_df` which also appear in `failures_df`
original_maint_df_len = len(maint_df)

# 1: Performing merge operation to include an indicator column
merged_df = pd.merge(
    left = maint_df,
    right = failures_df,
    on = maint_df.columns.tolist(),
    how = 'left',
    indicator = True
)

# 2: Filtering rows that are present in maint_df but not in failures_df
unique_to_maint_df = merged_df[
    merged_df['_merge'] == 'left_only'
]

# 3: Drop the '_merge' column as it's no longer needed
unique_to_maint_df = unique_to_maint_df.drop(
    '_merge',
    axis = 1
)

# 4: Overwriteing maint_df with the rows unique to it
maint_df = unique_to_maint_df

# print resulting updated `maint_df`
print(f"- {format(len(failures_df), ',')} total rows in failures_df.")
print(f"- {format(original_maint_df_len, ',')} total rows in original maint_df.")
print(f"- {format(len(maint_df), ',')} total rows in maint_df.")
maint_df.head(3)

- 761 total rows in failures_df.
- 3,286 total rows in original maint_df.
- 2,543 total rows in maint_df.


,datetime,machineID,comp
0,2014-06-01 06:00:00,1,comp2
1,2014-07-16 06:00:00,1,comp4
2,2014-07-31 06:00:00,1,comp3


In [ ]:
# Create dummy variables for the 'comp' column
comp_dummies = pd.get_dummies(maint_df['comp'])

# Rename the dummy variable columns immediately to indicate maintenance
comp_dummies = comp_dummies.rename(
    columns={
        'comp1': 'comp1_maint',
        'comp2': 'comp2_maint',
        'comp3': 'comp3_maint',
        'comp4': 'comp4_maint',
    }
)

# Concatenate the dummy variables back to the original DataFrame
maint_df = pd.concat([maint_df.drop(columns=['comp']), comp_dummies], axis=1)

# Now, group by 'machineID' and 'datetime' and sum to collapse each unique machineID-datetime pair
maint_df = maint_df.groupby(['machineID', 'datetime']).sum().reset_index()

# Since the renaming was done before the grouping, the grouped DataFrame should have the correct maintenance columns
# Proceed with further operations as necessary

In [ ]:
# sanity check: this should yield 1 row with multiple dummies set equal to 1
maint_df[
    (maint_df['datetime'] == '2015-01-05 06:00:00')
    & (maint_df['machineID'] == 1)
].head()

,machineID,datetime,comp1_maint,comp2_maint,comp3_maint,comp4_maint
4,1,2015-01-05 06:00:00,1,0,0,0


In [ ]:
tel_maint_df = pd.merge(
    left = tel_df,
    right = maint_df,
    on = ['machineID', 'datetime'],
    how = 'left'
)

# Filling null values in the maintenance columns with 0
tel_maint_df['comp1_maint'].fillna(0, inplace=True)
tel_maint_df['comp2_maint'].fillna(0, inplace=True)
tel_maint_df['comp3_maint'].fillna(0, inplace=True)
tel_maint_df['comp4_maint'].fillna(0, inplace=True)
print(f"- {format(len(tel_maint_df), ',')} total rows in tel_maint_df.")

for hour_window in HOUR_WINDOWS_LIST:
    nunique_step_ID = tel_maint_df[f'{hour_window}_step_ID'].nunique()
    print(f"- {format(nunique_step_ID, ',')} total unique step_ID values.")

    assert len(tel_maint_df) == nunique_step_ID

tel_maint_df.head(5)

- 876,100 total rows in tel_maint_df.
- 876,100 total unique step_ID values.
- 876,100 total unique step_ID values.
- 876,100 total unique step_ID values.
- 876,100 total unique step_ID values.


,datetime,machineID,volt,rotate,pressure,vibration,12_machineID_time_window,12_order_in_time_window,12_machine_example_ID,12_step_ID,24_machineID_time_window,24_order_in_time_window,24_machine_example_ID,24_step_ID,48_machineID_time_window,48_order_in_time_window,48_machine_example_ID,48_step_ID,72_machineID_time_window,72_order_in_time_window,72_machine_example_ID,72_step_ID,comp1_maint,comp2_maint,comp3_maint,comp4_maint
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",0.0,0.0,0.0,0.0
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",0.0,0.0,0.0,0.0
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",0.0,0.0,0.0,0.0
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,1,4,"(1, 1)","(1, 1, 4)",1,4,"(1, 1)","(1, 1, 4)",1,4,"(1, 1)","(1, 1, 4)",1,4,"(1, 1)","(1, 1, 4)",0.0,0.0,0.0,0.0
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,1,5,"(1, 1)","(1, 1, 5)",1,5,"(1, 1)","(1, 1, 5)",1,5,"(1, 1)","(1, 1, 5)",1,5,"(1, 1)","(1, 1, 5)",0.0,0.0,0.0,0.0


In [ ]:
for hour_window in HOUR_WINDOWS_LIST:

    # Step 1: Identify duplicated 'step_ID' values
    duplicated_step_IDs = tel_maint_df[
        tel_maint_df.duplicated(f'{hour_window}_step_ID', keep=False)
    ][f'{hour_window}_step_ID'].unique()

    # Step 2: Examine data for one of the duplicated 'step_ID' values
    if len(duplicated_step_IDs) > 0:
        print(len(duplicated_step_IDs))
        example_duplicated_step_ID = duplicated_step_IDs[0]
        duplicate_rows = tel_maint_df[tel_maint_df[f'{hour_window}_step_ID'] == example_duplicated_step_ID]
        print(duplicate_rows)
    else:
        print("No duplicated 'step_ID' values found.")


No duplicated 'step_ID' values found.
No duplicated 'step_ID' values found.
No duplicated 'step_ID' values found.
No duplicated 'step_ID' values found.


### 3. Load Errors File and Merge onto Merged DataFrame

In [ ]:
errors_df = pd.read_csv(
    ERRORS_F,
)
errors_df['machineID'] = errors_df['machineID'].astype(int)
errors_df['datetime'] = pd.to_datetime(errors_df['datetime'])
errors_df['errorID'] = errors_df['errorID'].astype(str)

print(f"- {format(len(errors_df), ',')} total rows in errors_df.")
errors_df.head(3)

- 3,919 total rows in errors_df.


,datetime,machineID,errorID
0,2015-01-03 07:00:00,1,error1
1,2015-01-03 20:00:00,1,error3
2,2015-01-04 06:00:00,1,error5


In [ ]:
# Create dummy variables for the 'comp' column
errors_dummies = pd.get_dummies(errors_df['errorID'])

# Concatenate the dummy variables back to the original DataFrame
errors_df = pd.concat(
    [errors_df.drop(columns=['errorID']), errors_dummies],
    axis=1
  )

# Now, group by 'machineID' and 'datetime' and sum to collapse each unique machineID-datetime pair
errors_df = errors_df.groupby(['machineID', 'datetime']).sum().reset_index()

# Since the renaming was done before the grouping, the grouped DataFrame should have the correct maintenance columns
# Proceed with further operations as necessary

# For demonstration, showing the updated DataFrame with dummy variables for a specific machineID-datetime
errors_df.head()


,machineID,datetime,error1,error2,error3,error4,error5
0,1,2015-01-03 07:00:00,1,0,0,0,0
1,1,2015-01-03 20:00:00,0,0,1,0,0
2,1,2015-01-04 06:00:00,0,0,0,0,1
3,1,2015-01-10 15:00:00,0,0,0,1,0
4,1,2015-01-22 10:00:00,0,0,0,1,0


In [ ]:
merged_df = pd.merge(
    left = tel_maint_df,
    right = errors_df,
    on = ['machineID', 'datetime'],
    how = 'left'
)

# Filling null values in the error columns with 0
merged_df['error1'].fillna(0, inplace=True)
merged_df['error2'].fillna(0, inplace=True)
merged_df['error3'].fillna(0, inplace=True)
merged_df['error4'].fillna(0, inplace=True)
merged_df['error5'].fillna(0, inplace=True)

print(f"- {format(len(merged_df), ',')} total rows in merged_df.")
#nunique_step_ID = merged_df['step_ID'].nunique()
#print(f"- {format(nunique_step_ID, ',')} total unique step_ID values.")

assert len(merged_df) == nunique_step_ID
merged_df.head(5)

- 876,100 total rows in merged_df.


,datetime,machineID,volt,rotate,pressure,vibration,12_machineID_time_window,12_order_in_time_window,12_machine_example_ID,12_step_ID,24_machineID_time_window,24_order_in_time_window,24_machine_example_ID,24_step_ID,48_machineID_time_window,48_order_in_time_window,48_machine_example_ID,48_step_ID,72_machineID_time_window,72_order_in_time_window,72_machine_example_ID,72_step_ID,comp1_maint,comp2_maint,comp3_maint,comp4_maint,error1,error2,error3,error4,error5
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,1,4,"(1, 1)","(1, 1, 4)",1,4,"(1, 1)","(1, 1, 4)",1,4,"(1, 1)","(1, 1, 4)",1,4,"(1, 1)","(1, 1, 4)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,1,5,"(1, 1)","(1, 1, 5)",1,5,"(1, 1)","(1, 1, 5)",1,5,"(1, 1)","(1, 1, 5)",1,5,"(1, 1)","(1, 1, 5)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
for hour_window in HOUR_WINDOWS_LIST:

    # Step 1: Identify duplicated 'step_ID' values
    duplicated_step_IDs = tel_maint_df[
        tel_maint_df.duplicated(f'{hour_window}_step_ID', keep=False)
    ][f'{hour_window}_step_ID'].unique()

    # Step 2: Examine data for one of the duplicated 'step_ID' values
    if len(duplicated_step_IDs) > 0:
        print(len(duplicated_step_IDs))
        example_duplicated_step_ID = duplicated_step_IDs[0]
        duplicate_rows = tel_maint_df[tel_maint_df[f'{hour_window}_step_ID'] == example_duplicated_step_ID]
        print(duplicate_rows)
    else:
        print("No duplicated 'step_ID' values found.")

No duplicated 'step_ID' values found.
No duplicated 'step_ID' values found.
No duplicated 'step_ID' values found.
No duplicated 'step_ID' values found.


### 4. Load Failures File and Merge onto Merged DataFrame

In [ ]:
failures_df = pd.read_csv(
    FAILURES_F,
)
failures_df['machineID'] = failures_df['machineID'].astype(int)
failures_df['datetime'] = pd.to_datetime(failures_df['datetime'])
failures_df['failure'] = failures_df['failure'].astype(str)

print(f"- {format(len(failures_df), ',')} total rows in failures_df.")
failures_df.head(3)

- 761 total rows in failures_df.


,datetime,machineID,failure
0,2015-01-05 06:00:00,1,comp4
1,2015-03-06 06:00:00,1,comp1
2,2015-04-20 06:00:00,1,comp2


In [ ]:
# Create dummy variables for the 'comp' column
comp_dummies = pd.get_dummies(failures_df['failure'])

# Rename the dummy variable columns immediately to indicate maintenance
comp_dummies = comp_dummies.rename(
    columns={
        'comp1': 'comp1_failure',
        'comp2': 'comp2_failure',
        'comp3': 'comp3_failure',
        'comp4': 'comp4_failure',
    }
)

# Concatenate the dummy variables back to the original DataFrame
failures_df = pd.concat(
    [failures_df.drop(columns = ['failure']), comp_dummies],
    axis = 1
)

# Now, group by 'machineID' and 'datetime' and sum to collapse each unique machineID-datetime pair
failures_df = failures_df.groupby(
    ['machineID', 'datetime']
).sum().reset_index()

# Since the renaming was done before the grouping, the grouped DataFrame should have the correct maintenance columns
# Proceed with further operations as necessary

failures_df.head()

,machineID,datetime,comp1_failure,comp2_failure,comp3_failure,comp4_failure
0,1,2015-01-05 06:00:00,0,0,0,1
1,1,2015-03-06 06:00:00,1,0,0,0
2,1,2015-04-20 06:00:00,0,1,0,0
3,1,2015-06-19 06:00:00,0,0,0,1
4,1,2015-09-02 06:00:00,0,0,0,1


In [ ]:
merged_df = pd.merge(
    left = merged_df,
    right = failures_df,
    on = ['machineID', 'datetime'],
    how = 'left'
)

# Filling null values in the error columns with 0
merged_df['comp1_failure'].fillna(0, inplace = True)
merged_df['comp2_failure'].fillna(0, inplace = True)
merged_df['comp3_failure'].fillna(0, inplace = True)
merged_df['comp4_failure'].fillna(0, inplace = True)

print(f"- {format(len(merged_df), ',')} total rows in merged_df.")
#nunique_step_ID = merged_df['step_ID'].nunique()
#print(f"- {format(nunique_step_ID, ',')} total unique step_ID values.")

assert len(merged_df) == nunique_step_ID
merged_df.head(5)

- 876,100 total rows in merged_df.


,datetime,machineID,volt,rotate,pressure,vibration,12_machineID_time_window,12_order_in_time_window,12_machine_example_ID,12_step_ID,24_machineID_time_window,24_order_in_time_window,24_machine_example_ID,24_step_ID,48_machineID_time_window,48_order_in_time_window,48_machine_example_ID,48_step_ID,72_machineID_time_window,72_order_in_time_window,72_machine_example_ID,72_step_ID,comp1_maint,comp2_maint,comp3_maint,comp4_maint,error1,error2,error3,error4,error5,comp1_failure,comp2_failure,comp3_failure,comp4_failure
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,1,4,"(1, 1)","(1, 1, 4)",1,4,"(1, 1)","(1, 1, 4)",1,4,"(1, 1)","(1, 1, 4)",1,4,"(1, 1)","(1, 1, 4)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,1,5,"(1, 1)","(1, 1, 5)",1,5,"(1, 1)","(1, 1, 5)",1,5,"(1, 1)","(1, 1, 5)",1,5,"(1, 1)","(1, 1, 5)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
for hour_window in HOUR_WINDOWS_LIST:

    # Step 1: Identify duplicated 'step_ID' values
    duplicated_step_IDs = merged_df[
        merged_df.duplicated(f'{hour_window}_step_ID', keep=False)
    ][f'{hour_window}_step_ID'].unique()

    # Step 2: Examine data for one of the duplicated 'step_ID' values
    if len(duplicated_step_IDs) > 0:
        print(len(duplicated_step_IDs))
        example_duplicated_step_ID = duplicated_step_IDs[0]
        duplicate_rows = merged_df[merged_df[f'{hour_window}_step_ID'] == example_duplicated_step_ID]
        print(duplicate_rows)
    else:
        print("No duplicated 'step_ID' values found.")


No duplicated 'step_ID' values found.
No duplicated 'step_ID' values found.
No duplicated 'step_ID' values found.
No duplicated 'step_ID' values found.


### 5. Load Machines File and Merge onto Merged DataFrame

In [ ]:
machines_df = pd.read_csv(
    MACHINES_F,
)
machines_df['machineID'] = machines_df['machineID'].astype(int)
machines_df['model'] = machines_df['model'].astype(str)
machines_df['age'] = machines_df['age'].astype(int)

print(f"- {format(len(machines_df), ',')} total rows in machines_df.")
machines_df.head(5)

- 100 total rows in machines_df.


,machineID,model,age
0,1,model3,18
1,2,model4,7
2,3,model3,8
3,4,model3,7
4,5,model3,2


In [ ]:
comp_dummies = pd.get_dummies(
    machines_df['model'],
    drop_first = True
)

# Concatenating the dummy variables back to the original DataFrame, excluding the original 'model' column
machines_df = pd.concat(
    [machines_df.drop(columns=['model']), comp_dummies],
    axis=1
)

# For demonstration, showing the updated DataFrame with dummy variables for a specific machineID-datetime
machines_df.head()

,machineID,age,model2,model3,model4
0,1,18,False,True,False
1,2,7,False,False,True
2,3,8,False,True,False
3,4,7,False,True,False
4,5,2,False,True,False


In [ ]:
merged_df = pd.merge(
    left = merged_df,
    right = machines_df,
    on = 'machineID'
)

print(f"- {format(len(merged_df), ',')} total rows in merged_df.")
merged_df.head(3)

- 876,100 total rows in merged_df.


,datetime,machineID,volt,rotate,pressure,vibration,12_machineID_time_window,12_order_in_time_window,12_machine_example_ID,12_step_ID,24_machineID_time_window,24_order_in_time_window,24_machine_example_ID,24_step_ID,48_machineID_time_window,48_order_in_time_window,48_machine_example_ID,48_step_ID,72_machineID_time_window,72_order_in_time_window,72_machine_example_ID,72_step_ID,comp1_maint,comp2_maint,comp3_maint,comp4_maint,error1,error2,error3,error4,error5,comp1_failure,comp2_failure,comp3_failure,comp4_failure,age,model2,model3,model4
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,False,True,False
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,False,True,False
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,False,True,False


In [ ]:
# formatting dummy columns
dummy_cols_list = [
    'comp1_maint','comp2_maint','comp3_maint','comp4_maint',
    'error1','error2','error3','error4','error5',
    'comp1_failure','comp2_failure','comp3_failure','comp4_failure',
]
for c in dummy_cols_list:
    merged_df[c] = merged_df[c].astype(int)

In [ ]:
merged_df.head(3)

,datetime,machineID,volt,rotate,pressure,vibration,12_machineID_time_window,12_order_in_time_window,12_machine_example_ID,12_step_ID,24_machineID_time_window,24_order_in_time_window,24_machine_example_ID,24_step_ID,48_machineID_time_window,48_order_in_time_window,48_machine_example_ID,48_step_ID,72_machineID_time_window,72_order_in_time_window,72_machine_example_ID,72_step_ID,comp1_maint,comp2_maint,comp3_maint,comp4_maint,error1,error2,error3,error4,error5,comp1_failure,comp2_failure,comp3_failure,comp4_failure,age,model2,model3,model4
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",1,1,"(1, 1)","(1, 1, 1)",0,0,0,0,0,0,0,0,0,0,0,0,0,18,False,True,False
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",1,2,"(1, 1)","(1, 1, 2)",0,0,0,0,0,0,0,0,0,0,0,0,0,18,False,True,False
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",1,3,"(1, 1)","(1, 1, 3)",0,0,0,0,0,0,0,0,0,0,0,0,0,18,False,True,False


### 6. Final Formatting Before Saving

In [ ]:
# formatting IDs so that they are padded with zeros up to 3 digits
merged_df['machineID'] = merged_df['machineID'].apply(
    lambda s: f'{s:03}'
)

for hour_window in HOUR_WINDOWS_LIST:
    merged_df[f'{hour_window}_machineID_time_window'] = merged_df[f'{hour_window}_machineID_time_window'].apply(
        lambda s: f'{s:03}'
    )
    merged_df[f'{hour_window}_order_in_time_window'] = merged_df[f'{hour_window}_order_in_time_window'].apply(
        lambda s: f'{s:03}'
    )

    # re-do keys using updated columns
    merged_df[f'{hour_window}_machine_example_ID'] = "(" + merged_df['machineID'].astype(str) + ", " + merged_df[f'{hour_window}_machineID_time_window'].astype(str) + ")"
    merged_df[f'{hour_window}_step_ID'] = "(" + merged_df['machineID'].astype(str) + ", " + merged_df[f'{hour_window}_machineID_time_window'].astype(str) + ", " + merged_df[f'{hour_window}_order_in_time_window'].astype(str) + ")"

In [ ]:
print(f"- {format(len(merged_df), ',')} total rows in merged_df.")
merged_df.head(5)

- 876,100 total rows in merged_df.


,datetime,machineID,volt,rotate,pressure,vibration,12_machineID_time_window,12_order_in_time_window,12_machine_example_ID,12_step_ID,24_machineID_time_window,24_order_in_time_window,24_machine_example_ID,24_step_ID,48_machineID_time_window,48_order_in_time_window,48_machine_example_ID,48_step_ID,72_machineID_time_window,72_order_in_time_window,72_machine_example_ID,72_step_ID,comp1_maint,comp2_maint,comp3_maint,comp4_maint,error1,error2,error3,error4,error5,comp1_failure,comp2_failure,comp3_failure,comp4_failure,age,model2,model3,model4
0,2015-01-01 06:00:00,001,176.217853,418.504078,113.077935,45.087686,001,001,"(001, 001)","(001, 001, 001)",001,001,"(001, 001)","(001, 001, 001)",001,001,"(001, 001)","(001, 001, 001)",001,001,"(001, 001)","(001, 001, 001)",0,0,0,0,0,0,0,0,0,0,0,0,0,18,False,True,False
1,2015-01-01 07:00:00,001,162.879223,402.747490,95.460525,43.413973,001,002,"(001, 001)","(001, 001, 002)",001,002,"(001, 001)","(001, 001, 002)",001,002,"(001, 001)","(001, 001, 002)",001,002,"(001, 001)","(001, 001, 002)",0,0,0,0,0,0,0,0,0,0,0,0,0,18,False,True,False
2,2015-01-01 08:00:00,001,170.989902,527.349825,75.237905,34.178847,001,003,"(001, 001)","(001, 001, 003)",001,003,"(001, 001)","(001, 001, 003)",001,003,"(001, 001)","(001, 001, 003)",001,003,"(001, 001)","(001, 001, 003)",0,0,0,0,0,0,0,0,0,0,0,0,0,18,False,True,False
3,2015-01-01 09:00:00,001,162.462833,346.149335,109.248561,41.122144,001,004,"(001, 001)","(001, 001, 004)",001,004,"(001, 001)","(001, 001, 004)",001,004,"(001, 001)","(001, 001, 004)",001,004,"(001, 001)","(001, 001, 004)",0,0,0,0,0,0,0,0,0,0,0,0,0,18,False,True,False
4,2015-01-01 10:00:00,001,157.610021,435.376873,111.886648,25.990511,001,005,"(001, 001)","(001, 001, 005)",001,005,"(001, 001)","(001, 001, 005)",001,005,"(001, 001)","(001, 001, 005)",001,005,"(001, 001)","(001, 001, 005)",0,0,0,0,0,0,0,0,0,0,0,0,0,18,False,True,False


In [ ]:
print(f"12 hrs -> {merged_df['12_machine_example_ID'].nunique()}")
print(f"24 hrs -> {merged_df['24_machine_example_ID'].nunique()}")
print(f"48 hrs -> {merged_df['48_machine_example_ID'].nunique()}")
print(f"72 hrs -> {merged_df['72_machine_example_ID'].nunique()}")

12 hrs -> 73100
24 hrs -> 36600
48 hrs -> 18300
72 hrs -> 12200


### 7. Save DataFrame as CSV

In [ ]:
SAVE_LOC = os.path.join(SAVE_DATA_DIR, f"preprocessed_data_variable_hour_window_cols.csv")
merged_df.to_csv(
    SAVE_LOC,
    index = False,
    quoting = csv.QUOTE_NONNUMERIC
)